# Bert For Summarization
Trong bài thực hành này ta sẽ làm quen với mô hình Bert cho bài toán Text Summarization. 

Download dữ liệu theo đường dẫn sau : https://drive.google.com/file/d/1TtHWc3UgszDv9Uonz5PQGcygA3UMHadJ/view?usp=sharing

## Bước 1: Cài đặt môi trường và thư viện

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Mã nguồn gốc của bài thực hành [tại đây](https://github.com/nlpyang/BertSum.git). Mã nguồn mà ta sử dụng đã fix một vài lỗi nhỏ để chạy được

Ta lấy phần dataset cho bài thực hành [tại đây](https://drive.google.com/file/d/1x0d61LP9UAN389YN00z0Pv-7jQgirVg6/view)

In [ ]:
#Lấy mã nguồn gốc
%cd /content/drive/MyDrive/NLP_VINAI/Lession14/
!git clone https://github.com/nlpyang/BertSum.git
%cd /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum

/content/drive/MyDrive/NLP_VINAI/Lession14
fatal: destination path 'BertSum' already exists and is not an empty directory.
/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum


Cài đặt các thư viện cần thiết

In [ ]:
!pip install torch==1.1.0 torchvision==0.3.0
!pip install pytorch_pretrained_bert
!pip install pyrouge
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Bước 2: Giải nén dữ liệu và quan sát một vài mẫu dữ liệu

### 2.1. Giải nén dữ liệu

In [ ]:
!unzip bertsum_data.zip -d /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data

Archive:  bertsum_data.zip
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.test.0.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.test.1.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.test.2.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.test.3.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.test.4.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.test.5.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.train.0.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.train.100.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.train.101.bert.pt  
  inflating: /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/bert_data/cnndm.train.102.ber

### 2.2. Quan sát dữ liệu

In [ ]:
import os
os.chdir("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum")

In [ ]:
import torch

def get_data(subset):
  assert subset in ["train", "test", "valid"]
  data = []
  for file in os.listdir("bert_data"):
    if subset in file:
      data_from_file = torch.load(os.path.join("bert_data", file))
      data.extend(data_from_file)
  return data

In [ ]:
train_data = get_data("train")

In [ ]:
print(f"Training data có số mẫu dữ liệu như sau: {len(train_data)}")
print(train_data[0].keys())
print("src:", train_data[0]["src"])
print("labels:", train_data[0]["labels"])
print("segs:", train_data[0]["segs"])
print("clss:", train_data[0]["clss"])
print("src_txt:", train_data[0]["src_txt"])
num_wrd_src = 0
for sent in train_data[0]["src_txt"]:
  num_wrd_src += len(sent.split())
print("Độ dài src_txt: ", num_wrd_src)
print("tgt_txt:", train_data[0]["tgt_txt"])
print("Độ dài tgt_txt: ", len(train_data[0]["tgt_txt"].split(" ")))

Training data có số mẫu dữ liệu như sau: 287084
dict_keys(['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt'])
src: [101, 2002, 1005, 1055, 2006, 1996, 2648, 2559, 1999, 1010, 1998, 1996, 3193, 1997, 13489, 13619, 6968, 4492, 17994, 3849, 2000, 2562, 2893, 11737, 5017, 1012, 102, 101, 1037, 2739, 3034, 5958, 2013, 2010, 2047, 7728, 1999, 8252, 3607, 2104, 9363, 5596, 2074, 2129, 11737, 2010, 16746, 3711, 2000, 2022, 1010, 2004, 1996, 28368, 5969, 2343, 10865, 2008, 2010, 3677, 1011, 1011, 1998, 4022, 27398, 1011, 1011, 2001, 2025, 2105, 1998, 2018, 2589, 2210, 2000, 2191, 2010, 2994, 2062, 6625, 1012, 102, 101, 1000, 1045, 5136, 2008, 3607, 2442, 1998, 2038, 2000, 2552, 1010, 1000, 13619, 6968, 4492, 17994, 2409, 1037, 6887, 7911, 26807, 1997, 12060, 2040, 2018, 9240, 1999, 1996, 2103, 1997, 20996, 29473, 1011, 2006, 1011, 2123, 1010, 2379, 1996, 8772, 3675, 2007, 5924, 1998, 2055, 6352, 2661, 2148, 1997, 4924, 1012, 102, 101, 2002, 2106, 2025, 20648, 2054, 4506, 2002, 2001, 5327, 

In [ ]:
# Xóa biến train_data để làm rỗng bộ nhớ
del train_data

### Phần 3: Huấn luyện mô hình và đánh giá

In [ ]:
%cd /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum
!mkdir -p /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints

/content/drive/MyDrive/NLP_VINAI/Lession14/bai14


In [ ]:
import torch
torch.cuda.get_device_name()

'Tesla P100-PCIE-16GB'

**Huấn luyện mô hình với option classifier**

In [ ]:
!mkdir -p  /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/classifier
!mkdir -p /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/classifier
!touch /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/classifier/results.log

In [ ]:
!python src/train.py -mode train -encoder classifier -dropout 0.1 \
-bert_data_path bert_data/cnndm \
-model_path checkpoints/classifier \
-lr 2e-3 \
-visible_gpus 0 \
-gpu_ranks 0  \
-report_every 50 \
-save_checkpoint_steps 500 \
-batch_size 3000 \
-decay_method noam \
-train_steps 1000 \
-accum_count 2 \
-log_file log/classifier/results.log \
-use_interval true \
-warmup_steps 10000 \

[2022-06-23 03:06:20,866 INFO] Device ID 0
[2022-06-23 03:06:20,869 INFO] Device cuda
[2022-06-23 03:06:21,296 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2022-06-23 03:06:21,297 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmplnf_4kgu
[2022-06-23 03:06:25,710 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2022-06-23 03:06:33,256 INFO] Summarizer(
  (bert): 

**Huấn luyện mô hình với option transformer**

In [ ]:
!mkdir -p  /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/transformer
!mkdir -p /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/transfomer
!touch /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/transfomer/results.log

In [ ]:
!python src/train.py -mode train -encoder transformer -dropout 0.1 \
-bert_data_path bert_data/cnndm \
-model_path checkpoints/transformer \
-gpu_ranks 0 -visible_gpus 0   \
-world_size 1 \
-report_every 50 \
-save_checkpoint_steps 100 \
-batch_size 3000 \
-decay_method noam \
-train_steps 500 \
-accum_count 2 \
-log_file log/transfomer/results.log \
-use_interval true \
-warmup_steps 10000 \
-ff_size 2048 \
-inter_layers 2 \
-heads 8

[2022-06-23 03:26:15,229 INFO] Device ID 0
[2022-06-23 03:26:15,229 INFO] Device cuda
[2022-06-23 03:26:15,613 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2022-06-23 03:26:15,614 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpns_1_rac
[2022-06-23 03:26:19,758 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2022-06-23 03:26:24,549 INFO] Summarizer(
  (bert): 

**Huấn luyện mô hình với option RNN**

In [ ]:
!mkdir -p /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/RNN
!mkdir -p /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/RNN
!touch /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/RNN/results.log

In [ ]:
!python src/train.py -mode train -encoder rnn  -dropout 0.1 \
-bert_data_path bert_data/cnndm \
-model_path checkpoints/RNN \
-lr 2e-3 \
-visible_gpus 0 \
-gpu_ranks 0 \
-world_size 1 \
-report_every 50 \
-save_checkpoint_steps 100 \
-batch_size 3000 \
-decay_method noam \
-train_steps 500 \
-accum_count 2 \
-log_file log/RNN/results.log \
-use_interval true \
-warmup_steps 10000 \
-rnn_size 768 \
-dropout 0.1 \


[2022-06-23 03:36:04,001 INFO] Device ID 0
[2022-06-23 03:36:04,002 INFO] Device cuda
[2022-06-23 03:36:04,403 INFO] loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
[2022-06-23 03:36:04,403 INFO] extracting archive file ../temp/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpd4jbrmke
[2022-06-23 03:36:08,546 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

[2022-06-23 03:36:13,322 INFO] Summarizer(
  (bert): 

Chạy đoạn code dưới đây để cài đặt package ROUGE để tính hiệu năng cho mô hình

In [ ]:
%cd /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum

/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum


In [ ]:
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/pyrouge/tools/ROUGE-1.5.5
!sudo cpan App::cpanminus
!sudo cpanm XML::DOM
!python pyrouge/setup.py install


os.chdir("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/pyrouge/tools/ROUGE-1.5.5/data/WordNet-2.0-Exceptions")
if os.path.isfile("./WordNet-2.0.exc.db"):
  os.remove("./WordNet-2.0.exc.db")
! ./buildExeptionDB.pl ./ ./smart_common_words.txt ./WordNet-2.0.exc.db

if os.path.isfile("../WordNet-2.0.exc.db"):
  os.remove("../WordNet-2.0.exc.db")

os.chdir("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/pyrouge/tools/ROUGE-1.5.5/data")
!ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

Cloning into 'pyrouge'...
remote: Enumerating objects: 393, done.
remote: Total 393 (delta 0), reused 0 (delta 0), pack-reused 393
Receiving objects: 100% (393/393), 298.74 KiB | 3.69 MiB/s, done.
Resolving deltas: 100% (109/109), done.
Checking out files: 100% (355/355), done.
2022-06-23 03:53:54,899 [MainThread  ] [INFO ]  Set ROUGE home directory to /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/pyrouge/tools/ROUGE-1.5.5.
Loading internal null logger. Install Log::Log4perl for logging messages
Reading '/root/.cpan/Metadata'
  Database was generated on Thu, 23 Jun 2022 03:17:02 GMT
App::cpanminus is up to date (1.7046).
XML::DOM is up to date. (1.46)
running install
running bdist_egg
running egg_info
creating pyrouge.egg-info
writing pyrouge.egg-info/PKG-INFO
writing dependency_links to pyrouge.egg-info/dependency_links.txt
writing requirements to pyrouge.egg-info/requires.txt
writing top-level names to pyrouge.egg-info/top_level.txt
writing manifest file 'pyrouge.egg-info/SOURCE

Đánh giá mô hình với option classifier

In [ ]:
!mkdir -p  /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/results/classifier

In [ ]:
os.chdir("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum")
!python src/train.py -mode validate \
-bert_data_path bert_data/cnndm -model_path /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/classifier -visible_gpus 0  -gpu_ranks 0 \
-batch_size 15000  -log_file /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/classifier/test_results.log -result_path /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/results/classifier/ \
-test_all -block_trigram true -bert_config_path bert_config_uncased_base.json 

[2022-06-23 03:59:07,396 INFO] Loading checkpoint from /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/classifier/model_step_500.pt
Namespace(accum_count=1, batch_size=15000, bert_config_path='bert_config_uncased_base.json', bert_data_path='bert_data/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='classifier', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/classifier/test_results.log', lr=1, max_grad_norm=0, mode='validate', model_path='/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/classifier', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/results/classifier/', rnn_size=512, save_checkpoint_steps=5, seed=666, temp_dir='temp', test_all=True, test_from='', train_from='', train_steps=1000, use_i

In [ ]:
with open("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/classifier/test_results.log", "r") as f:
  result = f.read()
print(result)

[2022-06-23 03:59:07,396 INFO] Loading checkpoint from /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/classifier/model_step_500.pt
[2022-06-23 03:59:18,181 INFO] Loading valid dataset from bert_data/cnndm.valid.0.bert.pt, number of examples: 2001
[2022-06-23 03:59:18,193 INFO] * number of parameters: 109483009
[2022-06-23 03:59:57,546 INFO] Loading valid dataset from bert_data/cnndm.valid.1.bert.pt, number of examples: 2001
[2022-06-23 04:00:36,365 INFO] Loading valid dataset from bert_data/cnndm.valid.2.bert.pt, number of examples: 2001
[2022-06-23 04:01:15,158 INFO] Loading valid dataset from bert_data/cnndm.valid.3.bert.pt, number of examples: 2000
[2022-06-23 04:01:54,233 INFO] Loading valid dataset from bert_data/cnndm.valid.4.bert.pt, number of examples: 2001
[2022-06-23 04:02:32,975 INFO] Loading valid dataset from bert_data/cnndm.valid.5.bert.pt, number of examples: 2001
[2022-06-23 04:03:11,803 INFO] Loading valid dataset from bert_data/cnndm.valid.6.bert.pt, n

Đánh giá mô hình với option transfomer

In [ ]:
!mkdir -p  /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/results/transformer

In [ ]:
os.chdir("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum")
!python src/train.py -mode validate \
-bert_data_path bert_data/cnndm -model_path /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/transformer -visible_gpus 0  -gpu_ranks 0 \
-batch_size 15000  -log_file /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/transfomer/test_results.log -result_path /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/results/transformer \
-test_all -block_trigram true -bert_config_path bert_config_uncased_base.json 

[2022-06-23 05:28:00,641 INFO] Loading checkpoint from /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/transformer/model_step_100.pt
Traceback (most recent call last):
  File "src/train.py", line 342, in <module>
    wait_and_validate(args, device_id)
  File "src/train.py", line 129, in wait_and_validate
    xent = validate(args,  device_id, cp, step)
  File "src/train.py", line 173, in validate
    checkpoint = torch.load(test_from, map_location=lambda storage, loc: storage)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 387, in load
    return _load(f, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 581, in _load
    deserialized_objects[key]._set_from_file(f, offset, f_should_read_directly)
KeyboardInterrupt


In [ ]:
with open("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/transfomer/test_results.log", "r") as f:
  result = f.read()
print(result)

[2022-06-23 04:15:00,740 INFO] Loading checkpoint from /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/transformer/model_step_100.pt
[2022-06-23 04:15:12,143 INFO] Loading valid dataset from bert_data/cnndm.valid.0.bert.pt, number of examples: 2001
[2022-06-23 04:15:12,152 INFO] * number of parameters: 120512513
[2022-06-23 04:15:51,307 INFO] Loading valid dataset from bert_data/cnndm.valid.1.bert.pt, number of examples: 2001
[2022-06-23 04:16:30,289 INFO] Loading valid dataset from bert_data/cnndm.valid.2.bert.pt, number of examples: 2001
[2022-06-23 04:17:09,242 INFO] Loading valid dataset from bert_data/cnndm.valid.3.bert.pt, number of examples: 2000
[2022-06-23 04:17:48,469 INFO] Loading valid dataset from bert_data/cnndm.valid.4.bert.pt, number of examples: 2001
[2022-06-23 04:18:27,367 INFO] Loading valid dataset from bert_data/cnndm.valid.5.bert.pt, number of examples: 2001
[2022-06-23 04:19:06,366 INFO] Loading valid dataset from bert_data/cnndm.valid.6.bert.pt, 

Đánh giá mô hình với option RNN

In [ ]:
!mkdir -p  /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/results/RNN

In [ ]:
os.chdir("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum")
!python src/train.py -mode validate \
-bert_data_path bert_data/cnndm -model_path /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/RNN -visible_gpus 0  -gpu_ranks 0 \
-batch_size 15000  -log_file /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/RNN/results.log -result_path   /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/results/RNN/ \
-test_all -block_trigram true -bert_config_path bert_config_uncased_base.json -temp_dir temp

[2022-06-23 04:56:43,341 INFO] Loading checkpoint from /content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/RNN/model_step_100.pt
Namespace(accum_count=1, batch_size=15000, bert_config_path='bert_config_uncased_base.json', bert_data_path='bert_data/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, encoder='rnn', ff_size=512, gpu_ranks=[0], heads=4, hidden_size=128, inter_layers=2, log_file='/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/RNN/results.log', lr=1, max_grad_norm=0, mode='validate', model_path='/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/checkpoints/RNN', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/results/RNN/', rnn_size=768, save_checkpoint_steps=5, seed=666, temp_dir='temp', test_all=True, test_from='', train_from='', train_steps=1000, use_interval=True, visible_gpus='0', warmup_s

In [ ]:
with open("/content/drive/MyDrive/NLP_VINAI/Lession14/BertSum/log/RNN/test_results.log", "r") as f:
  result = f.read()
print(result)

FileNotFoundError: ignored